# <div align="center">TVLA<div>



In [1]:
# import libraries
import os
import sys
import shutil
import json
import numpy as np
from foboslib import projmgr
from foboslib.ctrl import pynqctrl
from foboslib.ctrl.fobosctrl import FOBOSCtrl
from foboslib.dut import cw305dut
from foboslib.tvlaCalc import TVLACalc

#######################################################
# acquisition configuration
# connection to ctrl board
acqConf = {}
# Connection to Control Board
acqConf['ip'] = '192.168.10.98'
acqConf['port'] = 9995
# File names and locations
acqConf['workspace']       = "fobosworkspace"    # Folder to store projects
acqConf['projectName']     = "xoodyak-dom-1st"   # Name of this project. Files will be in workspace/projectName/
acqConf['dinFile']         = "dinFile.txt"       # test vectors in FOBOS format incl. plaintext and key, auto-generated
acqConf['fvrFile']         = "fvrchoicefile.txt" #
acqConf['cipherFile']      = "doutFile"          # observed output, result from applying test vectors
acqConf['traceFile']       = "powerTraces.npy"   # Name of file for recording power traces
# Cipher Specific Information
acqConf['bitFile']         = "aes_cw305.bit"     # Bitstream for programming the DUT, same directory as this notebook
acqConf['outlen']          = 88                  # size of a block of output in bytes
# Acquistion configuration
acqConf['traceNum']        = 100                 # number of traces to run
acqConf['samplingFreq']    = 50                  # sampling frequency of the Oscilloscope in Msps [range: 1 - 100]
acqConf['DUTClk']          = 10                  # clock frequency of the DUT in MHz [range: 1 - 100]
acqConf['samplesPerTrace'] = 5*80                # number of ADC samples in one trace
acqConf['ADCGain']         = 40                  # amplification of ADC input signal [range: 0 - 60]
acqConf['ADCHiLo']         = 1                   # 0 - low amplification, 1 - high amplification

In [3]:
# Configure project directories
pm = foboslib.projmgr.ProjectManager()
pm.setWorkSpaceDir(acqConf['workspace'])
pm.setProjName(acqConf['projectName'])
projDir = pm.getProjDir()

In [4]:
# connect to hardware and lock it ######################################
ctrl = foboslib.pynqctrl.PYNQCtrl(acqConf['ip'], acqConf['port'])
# Apply config to ctrl board ###########################################
try:
    # Apply settings to the Pynq Board
    ctrl.setDUTClk(acqConf['DUTClk'])                 # setting DUT clock resets all modules using this clock
    ctrl.setDUTInterface(FOBOSCtrl.INTERFACE_4BIT)    # Using Target connector -> INTERFACE_4BIT
    ctrl.setDUT(FOBOSCtrl.CW305)                      # Adjust Target connector pinout for DUT
    ctrl.setTriggerMode(FOBOSCtrl.TRG_FULL)           # Trigger is active during DUT operation
    ctrl.setOutLen(acqConf['outLen'])
    ctrl.setSamplingFrequency(acqConf['samplingFreq'])
    ctrl.setSamplesPerTrace(acqConf['samplesPerTrace'])
    ctrl.setADCGain(acqConf['ADCGain'])
    ctrl.setADCHiLo(acqConf['ADCHiLo'])

    ## configure DUT PRNG
    #ctrl.setOutLen(4)
#     seed = 0xaa11223344556677
#     num_rand_words = 1000
#     ctrl.confPrng(seed, num_rand_words)
    ##

except Exception as e:
    print(e)
    ctrl.disconnect()
else:
    # program DUT ##########################################################
    #dut = foboslib.nexys3dut.Nexys3DUT()
    #bitFile = os.path.join(projDir, acqConf['bitFile'])
    #dut.setBitFile(bitFile)
    #dut.program()
    # prepare i/o files ####################################################
    tvFileName = os.path.join(projDir, acqConf['dinFile'])
    #plainFileName = os.path.join(projDir, acqConf['plainFile'])
    tvFile = open(tvFileName, "r")
    captureDir = pm.getCaptureDir()
    cipherFileName = os.path.join(captureDir, acqConf['cipherFile'])
    cipherFile = open(cipherFileName, "w")
    traceFileName = os.path.join(captureDir, acqConf['traceFile'])
    traceFile = open(traceFileName, "a+b")
    shutil.copy(tvFileName, captureDir)
    fvrFileName = os.path.join(projDir, acqConf['fvrFile'])
    fvrFile = open(fvrFileName, 'r')
    shutil.copy(fvrFileName, captureDir)
    # save config to a file
    configFile = open(os.path.join(captureDir, 'acquisitionConfig.json'), "w")
    configFile.write(json.dumps(acqConf, indent=4))
    configFile.write(ctrl.config)
    print(ctrl.config)
    configFile.close()
    #t-test object
    tCalc = TVLACalc(acqConf['samplesPerTrace'])
    # Get traces############################################################
    print('Processing test vectors ...')
    traceNum = 0
    while traceNum < acqConf['traceNum']:
        data = tvFile.readline()
        status, result, trace = ctrl.processData2(data, acqConf['outLen'])
        cipherFile.write(result + "\n")
        np.save(traceFile, trace)
        c = fvrFile.read(1)
        tCalc.addTrace(trace, int(c))
        if traceNum % 10000 == 0:
            t_array, passed = tCalc.saveData1(captureDir, str(traceNum))
            sys.stdout.write('Progress:' + "{:.2f}".format(traceNum/acqConf['traceNum']*100) + '%\r')
            sys.stdout.flush()
        traceNum += 1

    tCalc.saveData1(captureDir, str(traceNum)) 
    print('Data acquisition complete.')
    ctrl.disconnect() # release and reset control board.
    traceFile.close()
    cipherFile.close()
    tvFile.close()
    fvrFile.close()

Acquired hardware lock
Successfully created new capture directory at ./xoodyak-dom/capture/attempt-02
# Acquisition parameters:
DUT_CLK = 16.0
OUT_LEN = 88
SAMPLING_FREQ = 80
ADC_GAIN = 30
SAMPLES_PER_TRACE = 400

Processing test vectors ...


/home/bakry/projects/GMU/Xoodyak-384-DOM/sca_verif/foboslib/tvlaCalc.py:67: RuntimeWarning: invalid value encountered in true_divide
  return self.CS2_0 / self.n_0


Data acquisition complete.
Released hardware lock.


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(trace)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
fig.patch.set_facecolor('white')
plt.rcParams.update({'font.size': 18})
traceFile = open(traceFileName, "r+b")
maxtrace = 100
plt.figure(figsize=(10,8))
plt.xlabel('Sample')
plt.ylabel('T-Value')
plt.title(f'TVLA {acqConf["projectName"]}')
for i in range(min(maxtrace, acqConf['traceNum'])):
    trace = np.load(traceFile)
    plt.plot(trace)

plt.savefig(os.path.join(captureDir, 'traces.png'),facecolor=fig.get_facecolor())
# plt.close()
traceFile.close()